# Kubernetes Autoscaling Demo with Kind
Project by Martin, Jan and Fynn

This notebook contains the step-by-step commands to set up and demonstrate
Kubernetes autoscaling using Kind (Kubernetes in Docker).

In [2]:
# Step 1: Set up Kind cluster
!kind create cluster --config three_node_config.yaml

Creating cluster "kind" ...
 ✓ Ensuring node image (kindest/node:v1.31.0) 🖼
 ✓ Preparing nodes 📦 📦 📦 7l
 ✓ Writing configuration 📜7l
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
 ✓ Joining worker nodes 🚜7l
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Thanks for using kind! 😊


In [3]:
# Step 2: Build and load Docker image
!docker build -t webapp:latest .
!kind load docker-image webapp:latest

[+] Building 0.0s (0/0)  docker:desktop-linux
[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.2s (2/3)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 227B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.9-slim         0.2s
[+] Building 0.3s (2/3)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 227B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                           

In [4]:
# Step 3: Apply Kubernetes configurations
!kubectl apply -f metrics-server.yaml
!kubectl apply -f webapp-deployment.yaml
!kubectl apply -f webapp-hpa.yaml

serviceaccount/metrics-server created
clusterrole.rbac.authorization.k8s.io/system:aggregated-metrics-reader created
clusterrole.rbac.authorization.k8s.io/system:metrics-server created
rolebinding.rbac.authorization.k8s.io/metrics-server-auth-reader created
clusterrolebinding.rbac.authorization.k8s.io/metrics-server:system:auth-delegator created
clusterrolebinding.rbac.authorization.k8s.io/system:metrics-server created
service/metrics-server created
deployment.apps/metrics-server created
apiservice.apiregistration.k8s.io/v1beta1.metrics.k8s.io created
deployment.apps/webapp created
service/webapp-service created
horizontalpodautoscaler.autoscaling/webapp-hpa created


In [5]:
# Step 4: Wait for pods to be ready
!kubectl wait --for=condition=Ready pods --all --timeout=300s

pod/webapp-6fc7996774-7fjcb condition met


In [6]:
# Step 5: Check the status of pods
!kubectl get pods

NAME                      READY   STATUS    RESTARTS   AGE
webapp-6fc7996774-7fjcb   1/1     Running   0          1s


In [7]:
# Step 6: Check the status of services
!kubectl get services

NAME             TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
kubernetes       ClusterIP   10.96.0.1       <none>        443/TCP        30s
webapp-service   NodePort    10.96.158.184   <none>        80:32275/TCP   2s


In [8]:
# Step 7: Set up port forwarding (run this in a separate terminal)
# !kubectl port-forward service/webapp-service 8080:80

In [9]:
# Step 8: Monitor HPA (run this in a separate terminal)
# !watch -n 1 kubectl get hpa webapp-hpa


In [10]:

# Step 9: Generate load (run this in a separate terminal)
# !ab -n 1000 -c 10 http://localhost:8080/
# => full load with max 10 pods

# !ab -n 1000 -c 3 http://localhost:8080/
# => 7 pods 30% cpu


In [11]:

# Step 10: Check the status of pods again to see scaling
!kubectl get pods

# Step 11: Check HPA status
!kubectl get hpa

# Step 12: Clean up 
#!kind delete cluster

NAME                      READY   STATUS    RESTARTS   AGE
webapp-6fc7996774-7fjcb   1/1     Running   0          2s
NAME         REFERENCE           TARGETS              MINPODS   MAXPODS   REPLICAS   AGE
webapp-hpa   Deployment/webapp   cpu: <unknown>/30%   1         10        0          3s
